In [ ]:
import param
import numpy as np
import holoviews as hv
hv.notebook_extension('bokeh', 'matplotlib')

HoloViews objects provide a convenient way of wrapping your data along with some metadata for exploration and visualization. In addition to the Elements and containers, HoloViews also provides so called ``Operations``, which can transform objects in custom ways allowing the user to build data processing pipelines. Examples of such operations are ``histogram``, ``rolling``, ``datashade`` or ``decimate``, which apply some computation on certain types of Element and return a new Element with the transformed data.

In this Tutorial we will discover how operations work, how to control their parameters and how to chain them. The [Dynamic_Operations](Dynamic_Operations.ipynb) extends what we have learned to demonstrate how operations can be applied lazily by using the ``dynamic`` flag, letting us define deferred processing pipelines that can drive highly complex visualizations and dashboards.


## Inspecting operations

The most common and useful kind of operations are the ``ElementOperation`` classes, which transform one Element or Overlay of Elements returning a new and transformed Element. All operations are so called ``ParameterizedFunction`` objects, which means that they allow defining parameters using the ``param`` library, providing validation for all parameters that control how the operation is applied. This also means we can change the parameters of an operation at the class-level, instance-level and when we call the operation. Let's start by having a look at the ``histogram`` operation. Just as for all other HoloViews objects we can inspect the ``histogram`` operation with the ``hv.help`` function:

In [ ]:
from holoviews.operation import histogram
hv.help(histogram)

## Applying operations

Above we can see a listing of all the parameters of the operation, with their defaults, the expected types and detailed docstrings for each one. The ``histogram`` operation can be applied to any Element and will by default generate a histogram for the first value dimension defined on the object it is applied to. As a simple example we can create an ``BoxWhisker`` Element containing samples from a normal distribution, and then apply the ``histogram`` operation to those samples in two ways: 1) by creating an instance on which we will change the ``num_bins`` and 2) by passing ``bin_range`` directly to the call of the operation:

In [ ]:
boxw = hv.BoxWhisker(np.random.randn(10000))
hist_instance = histogram.instance(num_bins=50)

boxw + hist_instance(boxw).relabel('num_bins=50') + histogram(boxw, bin_range=(0, 3)).relabel('bin_range=(0, 3)')

We can see that these two ways of using operations gives us convenient control over how the parameters are applied. An instance allows us to persist some defaults, while passing keyword argument to the operations applies the parameters for just that particular call to the operation.

As the name implies ``ElementOperations`` are applied to individual Elements. This means that even when you apply an operation to a container object containing multiple Elements of one type, e.g. ``NdLayout``, ``GridSpace`` and ``HoloMap`` containers, the operation is applied per Element, essentially mapping the operation over all Elements contained in the container object. As a simple example we can define a HoloMap of ``BoxWhisker`` Elements by varying the width of the distribution via the ``Sigma`` value and then apply the histogram operation to it:

In [ ]:
holomap = hv.HoloMap({(i*0.1+0.1): hv.BoxWhisker(np.random.randn(10000)*(i*0.1+0.1)) for i in range(5)},
                     kdims=['Sigma'])
holomap + histogram(holomap)

As you can see the operation has generated a ``Histogram`` for each value of ``Sigma`` in the ``HoloMap``. In this way we can apply the operation to the entire parameter space defined by a ``HoloMap``, ``GridSpace``, and ``NdLayout``.

## Combining operations

Since operations take a HoloViews object as input and return another HoloViews object we can very easily chain and combine multiple operations to perform complex analyses quickly and easily, while instantly visualizing the output.

In this example we'll work with a timeseries, so we'll define a small function to generate a random, noisy timeseries:

In [ ]:
from holoviews.operation import timeseries

def time_series(T = 1, N = 100, mu = 0.1, sigma = 0.1, S0 = 20):  
    """Parameterized noisy time series"""
    dt = float(T)/N
    t = np.linspace(0, T, N)
    W = np.random.standard_normal(size = N) 
    W = np.cumsum(W)*np.sqrt(dt) # standard brownian motion
    X = (mu-0.5*sigma**2)*t + sigma*W 
    S = S0*np.exp(X) # geometric brownian motion
    return S

curve = hv.Curve(time_series(N=1000))

Now we will start applying some operations to this data. HoloViews ships with two ready-to-use timeseries operations: the ``rolling`` operation, which applies a function over a rolling window, and a ``rolling_outlier_std`` operation that computes outlier points in a timeseries by excluding points less than ``sigma`` standard deviation removed from the rolling mean:

In [ ]:
%%opts Scatter [width=600] (color='black')
smoothed = curve * timeseries.rolling(curve) * timeseries.rolling_outlier_std(curve)
smoothed

## Defining custom operations

We can now define our own custom ElementOperation, if we wish. If you recall from above, operations accept both Elements and Overlays. This means we can define a simple operation that takes our Overlay of the original and smoothed Curve Elements and subtracts one from the other. Such a subtraction will give us the residual between the smoothed and unsmoothed Curves, removing long-term trends and leaving the short-term variation.

Defining an operation is very simple. An ElementOperation subclass should define a ``_process`` method, which accepts an ``element`` and an optional (and deprecated) ``key`` argument. Optionally we can also define parameters on the operation, which we can access using the ``self.p`` attribute on the operation. In this case we define a String-type parameter, which will become the name of the subtracted value dimension on the returned Element.

In [ ]:
from holoviews.operation import ElementOperation

class residual(ElementOperation):
    """
    Subtracts two curves from one another.
    """
    
    label = param.String(default='Residual', doc="""
        Defines the label of the returned Element.""")
    
    def _process(self, element, key=None):
        # Get first and second Element in overlay
        el1, el2 = element.get(0), element.get(1)
        
        # Get x-values and y-values of curves
        xvals = el1.dimension_values(0)
        yvals = el1.dimension_values(1)
        yvals2 = el2.dimension_values(1)
        
        # Return new Element with subtracted y-values
        # and new label
        return el1.clone((xvals, yvals-yvals2),
                         vdims=[self.p.label])

Having defined the residual operation let's try it out right away by applying it to our original and smoothed ``Curve``. We'll place the two objects on top of each other so they can share an x-axis and we can compare them directly:

In [ ]:
%%opts Curve [width=600] Overlay [xaxis=None]
(smoothed + residual(smoothed)).cols(1)

In this view we can immediately see that only a very small residual is left when applying this level of smoothing. However we have only tried one particular ``rolling_window`` value, the default value of ``10``. To assess how this parameter affects the residual we can evaluate the operation for a number of different values, as we do in the next section.

## Evaluating operation parameters

When applying an operation there are often various parameters to vary. Using traditional plotting methods it's often difficult to evaluate them interactively to get an detailed understanding of what they do. Here we will apply the ``rolling`` operations with varying ``rolling_window`` widths and ``window_type``s:

In [ ]:
rolled = hv.HoloMap({(w, str(wt)): timeseries.rolling(curve, rolling_window=w, window_type=wt)
                     for w in [10, 25, 50, 100, 200] for wt in [None, 'hamming', 'triang']},
                    kdims=['Window', 'Window Type'])
rolled

This view is already useful since we can compare the various parameter values. However since we can also chain operations we can also easily compute the residual and view the two together. To do so we simply overlay the ``HoloMap`` of smoothed curves on top of the original curve and again pass it to our new ``residual`` function. Then we again combine the the smoothed view with the original and see how the smoothing and residual vary with the operation parameters:

In [ ]:
%%opts Curve [width=600] Overlay [legend_position='top_left']
(curve(style=dict(color='black')) * rolled + residual(curve * rolled)).cols(1)

Using just a few more lines we have now evaluated the operation over a combination of different parameters providing a quick way to not only process data but gain a better understanding of both the parameters of the operation and the underlying data.

## Benefits of using operations

Now that we have seen some operations in action we can get some appreciation of what makes them useful. When working with data interactively you often end up doing a lot of data wrangling, which provides maximum flexibility but is neither reproducible nor maintainable. Operations allow you to encapsulate analysis code using a well defined interface that is well suited towards building complex analysis pipelines:

1. Their parameters are well defined by declaring parameters on the class. These parameters also perform validation on the types and ranges of the inputs.

2. Both the inputs and outputs of operations are visualizable, because the data **is** the visualization. This means you're not constantly context switching between data processing and visualization---you essentially get the visualization for free.

3. Operations understand HoloViews datastructures and can be applied to many Elements at once, allowing you to evaluate the operation with permutations of parameter values. This flexibility makes it easier to assess what the parameters of the operation are actually doing and how they shape your data.

4. As we will discover in the [Dynamic Operation Tutorial](Dynamic_Operations.ipynb), operations can be applied lazily to build up complex deferred data-processing pipelines, which can aid your data exploration and drive your interactive visualizations and dashboards.